In [ ]:
%%html
<style>
details {
  border: 2px solid #0D4864;
  border-radius: 4px;
  padding: 0.5em 0.5em 0;
  margin-bottom: 1em;
}

summary {
  font-weight: bold;
  margin: -0.5em -0.5em 0;
  padding: 10em;
  padding-top: 1em;
  color: #0D4864;
}

details[open] {
  padding: 0.5em;
  padding-bottom: 1em;
}

details > summary {
  padding: 4px 6px;
  background-color: #FFFFFF;
  border: none;
  box-shadow: 1px 1px 2px #0D4864;
  cursor: pointer;
}

details[open] > summary {
    margin-bottom: 1em;
}
</style>

# I. An introduction to the climap library

You recently constructed models and visualized predicted maps using the climap CLI. However, certain features, such as pre-processing, custom learning methods, or more extensive customizations, are not achievable through the CLI. To address this limitation, climap provides a library, which we will explore now. This tutorial aims to demonstrate the utilization of more features and capabilities of the `climap` package, with a specific emphasis on the technical aspects, employing only a simple simulated dataset as an illustrative example.

## Explore and prepare your data

The climap library internally employs the [pandas](https://pandas.pydata.org/) library for efficient data handling through dataframes. Prior to utilizing any functionality of the library, it is essential to load your data into a [pandas.DataFrame](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html). The dataframe should include, at a minimum, the target variable for prediction and the explanatory features as distinct columns. Additionally, each observation (row) must be appropriately labeled. Below is an example:

|OBS|E1 |E2 |E3 |P  |
|---|---|---|---|---|
|O1|1.5|0.0|9.6|1  |
|O2|0.2|2.2|0.5|1  |
|...|...|...|...|...|


The explanatory variables are denoted by E1 to E3, and the variable to predict is represented by the P column. Each observation is uniquely labeled from O1 to O2.

Here we will use [csv](https://en.wikipedia.org/wiki/Comma-separated_values) files as input data. The `pandas` library offers various features for loading data into dataframes from different sources. You can explore all the relevant features regarding input in the documentation: [pandas.io](https://pandas.pydata.org/docs/reference/io.html). Specifically for loading `csv` files, pandas provides the `read_csv` function, with detailed documentation: [pandas.read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html). 

💻 **Using the `read_csv` function, implement the code required to load the following files:**
* `observations.csv`
* `features.tsv`

<details>
<summary><b>💡 How to import the pandas package?</b><br></summary>

External python packages should be imported before being used:

<div class="alert alert-block alert-info" style="background-color: white; border: 0px solid; padding: 10px">

```python
import pandas as pd
```
</details>

<details>
<summary><b>💡 How to specify the csv separator?</b><br></summary>
You can specify the csv separator using the <code>sep</code> parameter.
</details>

<details>
<summary><b>💡 How to handle column headers?</b><br></summary>
By default, pandas infers the presence or absence of headers when loading data from file. If, for any reason, it does not work with your data, you have the option to utilize the <code>header</code>parameter.
</details>

<details>
<summary><b>💡 How to add labeled rows?</b><br></summary>
To ensure labeled observations, you must specificy the column serving as labels. This can be achieved using the <code>index_col</code> parameter.
</details>

<details>
<summary><b>✅ Solution</b><br></summary>

<div class="alert alert-block alert-info" style="background-color: white; border: 0px solid; padding: 10px">
    
```python
import pandas as pd
observations = pd.read_csv("./data/climap_lib/observations.tsv", sep="\t", index_col="Station")
env_features = pd.read_csv("./data/climap_lib/features.tsv", sep="\t")
```
</div>

</details>


In [ ]:
# The proposed training parameters have been chosen to enable you to train the models in a reasonable amount of time. 
# For example, it generally takes a large number of iterations to properly train a neural network, not just 1000. 
# As a result, sklearn can issue numerous warnings during execution. This code allows you to hide them so as not to pollute your output.
# You should run it once before any other cells.

import warnings
import os
warnings.filterwarnings("ignore")
os.environ['PYTHONWARNINGS'] = 'ignore'

To get an initial view of the data, it is possible to plot them on a map. For this, `climap` provides two functions:
* `climap.map.make_geodf(df: pd.DataFrame, latitude: str, longitude: str)`: This function allows to make a dataframe with geometric information for a subsequent plotting. The initial dataframe should contains two columns with latitude and longitude. The `latitude` and `longitude` parameters are the column labels in the dataframe.
* `climap.map.make_map(df: gpd.GeoDataFrame, col: str, **params)`: This function allows to produce a map from a dataframe with geometric information. The second parameter is the column to plot. Other parameters are the figure parameters forwarded to `matplotlib`. Ex: `fig = make_map(gdf, "PA", markersize=0.1)`

💻 **Using these functions, make**:
* A map representing the observations (presence or absence in each sampling station, columns `PA`).
* One map per environmental feature (named `F1`, `F2`, `F3`, `F4`, `F5` and `F6`).

<details>
<summary><b>💡 How to add a legend?</b><br></summary>

<div class="alert alert-block alert-info" style="background-color: white; border: 0px solid; padding: 10px">

```python
make_map(..., legend = True)
```
</details>

<details>
<summary><b>💡 How to add title and axis labels?</b><br></summary>

The map plot can use an external [matplotlib.Axes](https://matplotlib.org/stable/api/_as_gen/matplotlib.axes.Axes.html#matplotlib.axes.Axes).

<div class="alert alert-block alert-info" style="background-color: white; border: 0px solid; padding: 10px">

```python
fig, ax = plt.subplots(figsize=(10, 6))
ax.set_title("Title")
fig = make_map(..., legend=True, ax=ax)
```
</details>

<details>
<summary><b>💡 How to select the background map?</b><br></summary>

<div class="alert alert-block alert-info" style="background-color: white; border: 0px solid; padding: 10px">

```python
from climap.map import cx
fig = make_map(..., legend=True, ax=ax, source=cx.providers.CartoDB.DarkMatterNoLabels)

# To list all providers
print(list(cx.providers))
```
</details>

<details>
<summary><b>✅ Solution</b><br></summary>

<div class="alert alert-block alert-info" style="background-color: white; border: 0px solid; padding: 10px">
    
```python
from climap.map import make_geodf, make_map
observations_gdf = make_geodf(observations, latitude="Latitude", longitude="Longitude")
observations_fig = make_map(observations_gdf, "PA", markersize=0.1, legend=True)

env_features_gdf = make_geodf(env_features, latitude="Latitude", longitude="Longitude")

env_features_F1_fig = make_map(env_features_gdf, "F1", markersize=0.1, legend=True)
env_features_F2_fig = make_map(env_features_gdf, "F2", markersize=0.1, legend=True)
env_features_F3_fig = make_map(env_features_gdf, "F3", markersize=0.1, legend=True)
env_features_F4_fig = make_map(env_features_gdf, "F4", markersize=0.1, legend=True)
env_features_F5_fig = make_map(env_features_gdf, "F5", markersize=0.1, legend=True)
env_features_F6_fig = make_map(env_features_gdf, "F6", markersize=0.1, legend=True)
```
</div>

</details>

## 2. Pre-processing

Before proceeding with any training steps, we need to address several issues in the example data:

1. Here, we are focusing on classification. As evident from the input dataframe, the variable to predict, <code>PA</code>, represents abundances, so it needs to be binarized before training the models. 
2. Features measured at different scales may not contribute equally to model fitting and could introduce bias. In addition, some techniques perform better when features are on a similar scale. To mitigate this, it is common practice to normalize the features before model fitting.

In addition to leveraging `pandas` package for data handling, `climap` also relies on the well-known [scikit-learn]() library for the learning aspect, which is installed alongside it. The `sklearn` package provides the `sklearn.preprocessing` module containing various data transformers for normalization, you can find more details [here](https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing). For our specific case, we will use 3 types of transformers:

1. A binarizer (see [sklearn.preprocessing.Binarizer](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Binarizer.html#sklearn.preprocessing.Binarizer)). A binarizer simply transforms the data based on a specified threshold. Values lower than the threshold are mapped to 0, and values equal to or higher than the threshold are mapped to 1.
2. A standard scaler (see [sklearn.preprocessing.StandardScaler](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.StandardScaler.html#sklearn.preprocessing.StandardScaler)). Standard scaling centers the feature values at a mean of 0 with a standard deviation of 1, aligning the feature columns with the parameters of a standard normal distribution.
3. An one-hot encoder (see [sklearn.preprocessing.OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html)). You may notice that the values the column `F6` are discrete, but our models require numeric features. Here, we use a categorical encoder because our variable is not ordinal, meaning it does not have a natural order since it represents the ocean basin, as opposed to an ordinal variable (see [Ordinal encoding](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OrdinalEncoder.html)). For each category, the encoder produces a new binary column, filled with `1` if the row corresponds to the category, or `0` otherwise. 

The `sklearn` package also offers some utilities for easily combining and applying transformers. Notably, the [sklearn.compose.ColumnTransformer](https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn.compose.ColumnTransformer) automatically applies a set of transformers to a dataframe and outputs a newly transformed dataframe.

⚠️ By default, sklearn transformer pipelines do not output the result as `pandas` dataframes. You can instruct sklearn transformers to produce dataframes by using the following code:


```python
from sklearn import set_config
set_config(transform_output = "pandas")
```

💻 **Based on the information provided above, implement the code required to transform the dataframes: `observations` and `features`.
If you are comfortable with Python and interested, you have the option to implement the binarizer transformer yourself to explore creating a custom transformer in `sklearn`.**

<details>
<summary><b>💡 Some columns are missing in my transformed dataframe</b><br></summary>
By default, <a href="https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn.compose.ColumnTransformer">sklearn.compose.ColumnTransformer</a> discards all columns not specified for transformation. To retain them, you need to modify the value of the <code>remainder</code> parameter, as detailed in the documentation.
</details>

<details>
<summary><b>💡 Columns are renamed after transformation</b><br></summary>
By default, <a href="https://scikit-learn.org/stable/modules/generated/sklearn.compose.ColumnTransformer.html#sklearn.compose.ColumnTransformer">sklearn.compose.ColumnTransformer</a> renames transformed column names by adding a prefix corresponding to the transformer name. Take a look at the <code>verbose_feature_names_out</code> parameter.
</details>

<details>
<summary><b>💡 How to implement a custom transformer?</b><br></summary>
To construct a custom transformer, you can create a class that inherits from the base classes <a href="https://scikit-learn.org/stable/modules/generated/sklearn.base.BaseEstimator.html#sklearn-base-baseestimator">sklearn.base.BaseEstimator</a> and <a href="https://scikit-learn.org/stable/modules/generated/sklearn.base.TransformerMixin.html#sklearn.base.TransformerMixin">sklearn.TransformerMixin</a> and implement the <code>fit</code> and <code>transform</code> methods. The <code>fit</code> method, although required, is unnecessary in our case, therefore you can simply implement it with <code>return self</code>. The data transformation should take place within the <code>transform</code> method. Here is a template:
    
<div class="alert alert-block alert-info" style="background-color: white; border: 0px solid; padding: 10px">
    
```python
from sklearn.base import BaseEstimator, TransformerMixin
class CustomTransformerMulBy(BaseEstimator, TransformerMixin):
    def __init__(self, by):
        self.by = by

    def fit(self, X=None, y=None):
        return self

    def transform(self, X, y=None):
        cols = y if y else list(X)  # If y is None, apply the transformation to all columns.
        X[cols] = X[cols] * self.by
        return X
```
</div>

</details>

<details>
<summary><b>✅ Solution</b><br></summary>

<div class="alert alert-block alert-info" style="background-color: white; border: 0px solid; padding: 10px">
    
```python
from sklearn.preprocessing import Binarizer, StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn import set_config

from sklearn import set_config
set_config(transform_output = "pandas")

class CustomBinarizer(BaseEstimator, TransformerMixin):
    def __init__(self, threshold):
        self.threshold = threshold
    
    def fit(self, X=None, y=None):
        return self

    def transform(self, X, y=None):
        cols = y if y else list(X)
        X[cols] = (X[cols] > self.threshold).astype(int)
        return X

ct = ColumnTransformer(
    remainder = 'passthrough',
    transformers = [
        ("bin", CustomBinarizer(threshold=0), ["PA"]),
        ("mm", OneHotEncoder(sparse_output=False), ["F6"]),
        ("sd", StandardScaler(), ["F1", "F2", "F3", "F4", "F5"])
    ],
    verbose_feature_names_out = False
)

ct_env = ColumnTransformer(
    remainder = 'passthrough',
    transformers = [
        ("mm", OneHotEncoder(sparse_output=False), ["F6"]),
        ("sd", StandardScaler(), ["F1", "F2", "F3", "F4", "F5"])
    ],
    verbose_feature_names_out = False
)
observations = ct.fit_transform(observations)
env_features = ct_env.fit_transform(env_features)
```
</div>

</details>

## 3. Train a model


Before commencing model training, certain configurations need to be set. Model fitting is a non-deterministic process as it involves some randomness. Therefore, it is a best practice to establish a random seed before training to ensure reproducibility. Sklearn do not use its own random seed but relies on NumPy one. You can set it using the [np.random.seed]() function. Additionally, throughout the process, a string variable containing the name of the variable to predict, as well as a list of the names of explanatory features, will be required.

💻 **Set the random seed using NumPy's function, and create one variable containing the name of the variable to predict, and a list containing the names of the features.**

<details>
<summary><b>💡 How to retrieve feature column names?</b><br></summary>

Column names can be collected by calling `list` on a dataframe.
```python
names = list(df)
```
<br>
⚠️ Some columns should not be used for training, remember to remove them from the list.
</details>


<details>
<summary><b>✅ Solution</b><br></summary>

<div class="alert alert-block alert-info" style="background-color: white; border: 0px solid; padding: 10px">
    
```python
import numpy as np
np.random.seed(42)

to_predict = "PA"
features = list(observations)[1:-2]
```
</div>

</details>

### A. Basic fitting

Now that you have a dataframe prepared for training, you are ready to train your first model. The initial model you will train is a `RandomForest` classifier, available in `climap.model.classifiers`. Within climap, all models are constructed and utilized in a uniform manner. The only variables that may differ are those specific to each method. To instantiate a model class, the required parameters are the dataframe, the list of feature names, and the label corresponding to the variable to predict. Additionally, you can provide any additional parameters supported by the [underlying scikit-learn model](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier). The constructor signature is `Model(df: pd.DataFrame, x: List[str], y: str, **params)`, where `x` is a list of labels corresponding to features and `y` the label of the variable to predict. Once the model is constructed, it can be fitted using the `Model.fit()` method.

After model training, you can make predictions on new data using the `predict` and `predict_proba` methods. The function signatures are as follows:
* `predict(df: pd.DataFrame, features: List[str]) -> pd.DataFrame` returns a new dataframe with predictions included as a new column, suffixed with `_predicted`.
* `predict_proba(df: pd.DataFrame, features: List[str]) -> pd.DataFrame` returns a new dataframe with one new column per category with the predicted probabilities.


💻 **Construct the RandomForest classifier using your dataframe. Your forest should use 50 trees. Refer to the [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html#sklearn.ensemble.RandomForestClassifier) to find the right parameter.**  
💻 **Predict the presence/absence pattern for all observations.**  
💻 **Plot the predictions on a map**

<details>
<summary><b>✅ Solution</b><br></summary>

<div class="alert alert-block alert-info" style="background-color: white; border: 0px solid; padding: 10px">
    
```python
from climap.model.classifiers import RandomForest

rf = RandomForest(observations, features, to_predict, n_estimators=50).fit()
predictions_rf = rf.predict(env_features, features)
predictions_rf_gdf = make_geodf(predictions_rf, latitude="Latitude", longitude="Longitude")
predictions_rf_fig = make_map(predictions_rf_gdf, "PA_predicted", markersize=0.1, legend=True)
```
</div>

</details>

💻 **Predict the probabilities of presence for all observations.**  
💻 **Plot the probabilities on a map**

<details>
<summary><b>✅ Solution</b><br></summary>

<div class="alert alert-block alert-info" style="background-color: white; border: 0px solid; padding: 10px">
    
```python
predictions_rf_probs = rf.predict_proba(env_features, features)
predictions_rf_probs_gdf = make_geodf(predictions_rf_probs, latitude="Latitude", longitude="Longitude")
predictions_rf_probs_fig = make_map(predictions_rf_probs_gdf, 1, markersize=0.1, legend=True)
```
</div>

</details>

### B. Hyperparameters tuning

Hyperparameters play a significant role in determining the quality of your models. One approach to finding the best hyperparameters is to try multiple combinations of values and evaluate each combination. The idea is to provide what we call a parameter grid containing all parameters to test. For each combination, the input data is split into a training set and a test set, the model is trained, and evaluated using a scoring function. This process is known as cross-validation. In the end, we select the model with the best performances across all models we evaluated.

The tuning can be done using the `model.grid_tuning` method:

```python
m = Model(observations, features, to_predict)
m.grid_tuning(grid = {
    "params": 1 ,
}, scoring = "")
```

The `grid` parameter is a dictionary containing the hyperparameter values to test, and `scoring` is the scoring method used to evaluate the performances. Supported values for `scoring` are listed in the [sklearn documentation](https://scikit-learn.org/stable/modules/model_evaluation.html).

💻 **Construct a new RandomForest and tune it using a grid, you should try 100, 200, 300 and 400 for the number of trees. The scoring function is simply the accuracy.**    
💻 **What is the best number of trees? (Look at `model.report`)**  
💻 **What is the accuracy of the best model? (Look at `model.score`)**  

<details>
<summary><b>✅ Solution</b><br></summary>

<div class="alert alert-block alert-info" style="background-color: white; border: 0px solid; padding: 10px">
    
```python
rf_grid = RandomForest(observations, features, to_predict)
rf_grid.grid_tuning(grid = {
    "n_estimators": [100, 200, 300, 400],
}, scoring = "accuracy")

print(rf_grid.report)
print(rf_grid.score)
```
</div>

</details>

### C. Custom cross-validation

By default, the grid tuning process employs a *k*-fold cross-validation strategy. In brief, *k*-fold cross-validation shuffles the input data and divides it into *k* groups. Each group is then used as a test dataset while the remaining groups are used as training data. The model performance is evaluated at each iteration using the scoring function. The cross-validation procedure employed by the grid tuning can be changed by providing a custom cross-validation splitter to the 'cv' parameter. 

Several splitters are available in `sklearn`, see [splitter classes](https://scikit-learn.org/stable/modules/classes.html#splitter-classes). Additionally, you have the option to create your own [custom splitter](https://scikit-learn.org/stable/glossary.html#term-CV-splitter).

💻 **Perform the grid tuning with the following cross-validation strategy: Random cv with 10 splits and a Train/Test ratio of 90/10. Refer to the documentation to find the right splitter class.**  

<details>
<summary><b>✅ Solution</b><br></summary>

<div class="alert alert-block alert-info" style="background-color: white; border: 0px solid; padding: 10px">
    
```python
from sklearn.model_selection import ShuffleSplit

s = ShuffleSplit(n_splits=10, test_size=0.10)
rf_cv = RandomForest(observations, features, to_predict)
rf_cv.grid_tuning(grid = {
    "n_estimators": [100, 200, 300, 400],
}, scoring = "accuracy", cv = s)

print(rf_cv.report)
print(rf_cv.score)
print(rf_cv.params)
```
</div>

</details>

## 5. Ensemble learning

### A. Voting

Ensemble voting is supported through the `climap.model.EnsembleVoting`:
* `EnsembleVoting(df: pd.DataFrame, x: list, y: str, estimators: list, **params)`, where `estimators` is a list of tuple `name`:`model` and `params` are forwaded to the corresponding sklearn class: `VotingClassifier`.

Example:
```python
m1 = RandomForest(...)
m2 = GradientBoosting(...)

e = EnsembleVoting(df, x, y, estimators = [
    ("m1", m1)
    ("m2", m2)
], voting = "soft")
```

`EnsembleVoting` also supports the `grid_tuning` method to perform the hyperparameters grid tuning. Note that, the parameter in the grid should be prefixed with estimator names: `{name}__{param}`. 

Example:

```python
grid = {
    "m1__n_estimators": [100, 200]
}
e.grid_tuning(grid = grid)
```

💻 **Combine RandomForest, NeuralNet and GradientBoosting with ensemble voting ("soft" voting). Using random cv, 5 splits and a 90/10 ratio and the following grid:**

* RandomForest:
  * 100, 200, 300 and 400 trees
* NeuralNet:
  * 2 hidden layers with 2 neurons and 3 hidden layers with 3 neurons.
  * `lbfgs` or `adam` solver
  * Fixed number of iterations: 150 (to reduce learning time)
* GradientBoosting:
  * 100 and 200 trees
  * learning rate of 0.001 and 0.01


<details>
<summary><b>✅ Solution</b><br></summary>

<div class="alert alert-block alert-info" style="background-color: white; border: 0px solid; padding: 10px">
    
```python
from climap.model.classifiers import RandomForest, NeuralNet, GradientBoosting
from climap.model import EnsembleVoting

rf = RandomForest(observations, features, to_predict)
nn = NeuralNet(observations, features, to_predict, max_iter=150)
gb = GradientBoosting(observations, features, to_predict)

grid = {
    "rf__n_estimators": [100, 200, 300, 400],
    "nn__hidden_layer_sizes": [(2, 2), (3, 3)],
    "nn__solver": ["lbfgs", "adam"],
    "gb__learning_rate": [0.001, 0.01],
    "gb__n_estimators": [100, 200],
}

ev = EnsembleVoting(observations, features, to_predict, estimators=[
    ("rf", rf),
    ("nn", nn),
    ("gb", gb)
], voting="soft")

s = ShuffleSplit(n_splits=5, test_size=0.10)
ev.grid_tuning(grid = grid, cv = s, n_jobs = 8)
```
</div>

</details>



💻 **Predict the probabilities of presence for all observations.**  
💻 **Plot the probabilities on a map.**  
💻 **Compare performances with the model that only uses a `RandomForest`.**

<details>
<summary><b>✅ Solution</b><br></summary>

<div class="alert alert-block alert-info" style="background-color: white; border: 0px solid; padding: 10px">
    
```python
predictions_ev_probs = ev.predict_proba(env_features, features)
predictions_ev_probs_gdf = make_geodf(predictions_ev_probs, latitude="Latitude", longitude="Longitude")
predictions_ev_probs_fig = make_map(predictions_ev_probs_gdf, 1, markersize=0.1, legend=True)
```
</div>

</details>

### B. Stacking

Stacking works in the same way as voting but uses the `climap.model.EnsembleStacking` class:
* `EnsembleStacking(df: pd.DataFrame, x: list, y: str, estimators: list, final, **params)`, where `final` is the final estimator.

Example:
```python
m1 = RandomForest(...)
m2 = GradientBoosting(...)
f = RandomForest(...)

e = EnsembleStacking(df, x, y, estimators = [
    ("m1", m1)
    ("m2", m2)
], final = f)
```

Note that the prefix for the hyperparameters the final estimator is `final_estimator`.

💻 **Produce a model using stacking with the same methods and hyperparameters as for voting. The final estimator should be a RandomForest, with 100 or 300 trees.**

<details>
<summary><b>✅ Solution</b><br></summary>

<div class="alert alert-block alert-info" style="background-color: white; border: 0px solid; padding: 10px">
    
```python
from climap.model.classifiers import RandomForest, NeuralNet, GradientBoosting
from climap.model import EnsembleStacking

rf = RandomForest(observations, features, to_predict)
nn = NeuralNet(observations, features, to_predict, max_iter=1000)
gb = GradientBoosting(observations, features, to_predict)
final = RandomForest(observations, features, to_predict)

es = EnsembleStacking(
        observations, 
        features,
        to_predict,
        estimators=[
            ("rf", rf), 
            ("nn", nn), 
            ("gb", gb)
        ],
        final = final,
        passthrough = True 
)
# When passthrough = True, the final estimator is trained on the original data 
# and the predictions. When passthrough = False, only the predictions   
# are used by the final estimator


grid = {
    "rf__n_estimators": [100, 200, 300, 400],
    "nn__hidden_layer_sizes": [(2, 2), (3, 3)],
    "nn__solver": ["lbfgs", "adam"],
    "gb__learning_rate": [0.001, 0.01],
    "gb__n_estimators": [100, 200],
    "final_estimator__n_estimators": [100, 300]
}
s = ShuffleSplit(n_splits=5, test_size=0.10)
es.grid_tuning(grid = grid, scoring = "accuracy", cv = s, n_jobs = 8)
```

</div>

</details>


💻 **Predict the probabilities of presence for all observations.**  
💻 **Plot the probabilities on a map**
<details>
<summary><b>✅ Solution</b><br></summary>

<div class="alert alert-block alert-info" style="background-color: white; border: 0px solid; padding: 10px">
    
```python
predictions_es_probs = es.predict_proba(env_features, features)
predictions_es_probs_gdf = make_geodf(predictions_es_probs, latitude="Latitude", longitude="Longitude")
predictions_es_probs_fig = make_map(predictions_es_probs_gdf, 1, markersize=0.1, legend=True)
```
</div>

</details>

### C. Custom models

As mentioned earlier, climap is simply a wrapper around scikit-learn, meaning that every learning method from scikit-learn is easily integrable. Currently, there are three built-in classifiers, but you can use any other classifiers. To integrate a new classifier, you need to implement a new class that inherits from `climap.model.Model` as described in the example below:

```python
from climap.model import Model
from sklearn import Example

class MyExample(Model):
    def __init__(self, df: pd.DataFrame, x: list, y: str, **params) -> None:
        super().__init__(Example, df, x, y, **params)
```

💻 **Based on the example above, implement the support for [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html#sklearn.tree.DecisionTreeClassifier)**.  
💻 **Look at the supported hyperparameters in the sklearn documentation, and try to obtain a good predictor using grid tuning.**

Note that methods from other libraries may also be supported. The procedure will be described in the documentation when `climap` goes online.

<details>
<summary><b>✅ Solution</b><br></summary>

<div class="alert alert-block alert-info" style="background-color: white; border: 0px solid; padding: 10px">
    
```python
from sklearn.tree import DecisionTreeClassifier
from climap.model import Model

class DT(Model):
    def __init__(self, df: pd.DataFrame, x: list, y: str, **params) -> None:
        super().__init__(DecisionTreeClassifier, df, x, y, **params)

dt = DT(observations, features, to_predict).fit()
```
</div>

</details>

💻 **Bonus: Perform the Bathycoccus analysis using the library**